In [ ]:
import pandas as pd
df=pd.read_csv("/content/EURUSD_M15_202501012300_202506181730.csv",sep="\t")
df.head()

,<DATE>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<TICKVOL>,<VOL>,<SPREAD>
0,2025.01.01,23:00:00,1.03514,1.03526,1.03486,1.03493,44,0,29
1,2025.01.01,23:15:00,1.03493,1.03541,1.03490,1.03536,181,0,44
2,2025.01.01,23:30:00,1.03536,1.03567,1.03536,1.03565,376,0,24
3,2025.01.01,23:45:00,1.03563,1.03571,1.03550,1.03552,413,0,24
4,2025.01.02,00:00:00,1.03552,1.03566,1.03492,1.03539,443,0,1


In [ ]:
threshold_date = "2025.05.15"
train_df = df[df["<DATE>"] < threshold_date]
test_df = df[df["<DATE>"] >= threshold_date]

In [ ]:
train_df.drop(columns=["<DATE>","<TIME>","<VOL>","<SPREAD>"])
test_df.drop(columns=["<DATE>","<TIME>","<VOL>","<SPREAD>"])

,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<TICKVOL>
9115,1.11813,1.11823,1.11759,1.11793,511
9116,1.11792,1.11898,1.11778,1.11884,545
9117,1.11884,1.11943,1.11860,1.11873,618
9118,1.11873,1.11932,1.11858,1.11915,629
9119,1.11915,1.11964,1.11907,1.11956,677
...,...,...,...,...,...
11485,1.15116,1.15139,1.15065,1.15067,613
11486,1.15067,1.15067,1.15015,1.15030,457
11487,1.15031,1.15092,1.15029,1.15055,378
11488,1.15055,1.15068,1.15011,1.15026,382


In [ ]:
train_df =train_df.rename(columns={
    '<OPEN>': 'open',
    '<HIGH>': 'high',
    '<LOW>': 'low',
    '<CLOSE>': 'close',
    '<TICKVOL>': 'tick_volume'
})
test_df =test_df.rename(columns={
    '<OPEN>': 'open',
    '<HIGH>': 'high',
    '<LOW>': 'low',
    '<CLOSE>': 'close',
    '<TICKVOL>': 'tick_volume'
})

In [ ]:
from sklearn.preprocessing import MinMaxScaler

feature_cols = ['open','high','low','close','tick_volume']

scaler = MinMaxScaler()
scaler.fit(train_df[feature_cols])
train_scaled = scaler.transform(train_df[feature_cols])
test_scaled = scaler.transform(test_df[feature_cols])


In [ ]:
import numpy as np

def make_sequences(data_scaled, raw_df, lookback=10, target_offset=15):
    X, y = [], []
    for i in range(len(data_scaled)-lookback-target_offset+1):
        X.append(data_scaled[i:i+lookback])
        target_idx = i + lookback + target_offset - 1
        future_high = raw_df['high'].iloc[target_idx]
        future_low = raw_df['low'].iloc[target_idx]
        y.append([future_high, future_low])  # two targets
    return np.array(X), np.array(y)

X_train, y_train = make_sequences(train_scaled, train_df)
X_test, y_test = make_sequences(test_scaled, test_df)


In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv1D, LSTM, Dense, Dropout

model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(10, len(feature_cols))),
    Dropout(0.2),
    LSTM(64),
    Dense(32, activation='relu'),
    Dense(2)  # predict [future_high, future_low]
])
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 8, 64)          │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 8, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 36,194 (141.38 KB)

 Trainable params: 36,194 (141.38 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=50,
    batch_size=32
)

# Save model & scaler
model.save('scalping_model_high_low.h5')
import pickle
with open('scaler.pkl','wb') as f:
    pickle.dump(scaler, f)


Epoch 1/50
285/285 ━━━━━━━━━━━━━━━━━━━━ 7s 13ms/step - loss: 0.2678 - mae: 0.3150 - val_loss: 7.3202e-05 - val_mae: 0.0070
Epoch 2/50
285/285 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 1.2233e-04 - mae: 0.0086 - val_loss: 1.6430e-05 - val_mae: 0.0032
Epoch 3/50
285/285 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 7.8518e-05 - mae: 0.0069 - val_loss: 1.3860e-05 - val_mae: 0.0028
Epoch 4/50
285/285 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 5.3894e-05 - mae: 0.0057 - val_loss: 1.6134e-05 - val_mae: 0.0033
Epoch 5/50
285/285 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - loss: 4.0887e-05 - mae: 0.0050 - val_loss: 1.1164e-05 - val_mae: 0.0027
Epoch 6/50
285/285 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - loss: 3.5333e-05 - mae: 0.0047 - val_loss: 1.4387e-05 - val_mae: 0.0031
Epoch 7/50
285/285 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 3.2044e-05 - mae: 0.0044 - val_loss: 1.1261e-05 - val_mae: 0.0026
Epoch 8/50
285/285 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 2.8380e-05 - mae: 0.0042 - val_loss: 3.1451e-05 - val_mae: 0

In [ ]:
signal_df[signal_df["signal"]=="BUY"]

,date_index,close,future_high_pred,future_low_pred,signal
49,9173,1.11781,1.120739,1.121383,BUY
145,9269,1.11582,1.119556,1.120320,BUY
146,9270,1.11549,1.119009,1.119712,BUY
147,9271,1.11561,1.118354,1.119007,BUY
148,9272,1.11435,1.117648,1.118299,BUY
150,9274,1.11402,1.116296,1.117045,BUY
614,9738,1.13214,1.134662,1.135377,BUY
615,9739,1.13089,1.134073,1.134655,BUY
949,10073,1.12105,1.128121,1.128917,BUY
950,10074,1.12297,1.127305,1.127995,BUY


In [ ]:
import numpy as np
import pandas as pd

lookback = 10


test_scaled = scaler.transform(test_df[feature_cols]) # ensure using feature_cols


X_test, y_test = make_sequences(test_scaled, test_df)


y_pred = model.predict(X_test)


74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


In [ ]:
from sklearn.metrics import mean_absolute_error

# MAE for future_high
mae_high = mean_absolute_error(y_test[:,0], y_pred[:,0])

# MAE for future_low
mae_low = mean_absolute_error(y_test[:,1], y_pred[:,1])

# Combined average MAE
mae_avg = (mae_high + mae_low) / 2

print(f"MAE future high: {mae_high:.6f}")
print(f"MAE future low: {mae_low:.6f}")
print(f"MAE average: {mae_avg:.6f}")


MAE future high: 0.002067
MAE future low: 0.002192
MAE average: 0.002130


**Reasoning**:
Generate scatter plots to visualize the actual vs predicted future high and low prices on the test set as instructed, using subplots for clarity and adding appropriate titles and labels.



In [ ]:
import numpy as np
import pandas as pd

lookback = 10


test_scaled = scaler.transform(test_df[feature_cols]) # ensure using feature_cols


X_test, y_test = make_sequences(test_scaled, test_df)


y_pred = model.predict(X_test)

